### Cross-modal prediction

Objective: use the validation set for a pre-trained model in the cross mdoal setting
 - Load data
 - Load pre-trained model
 - Perform cross-modal prediction
 - Show example cell

In [1]:
import sys
from pathlib import Path
#Add path to parent folder for imports
sys.path.append(str(Path.cwd().parent))

In [2]:
import tensorflow as tf
from utils.dataset import load_bioarxiv_dataset,partitions
from model import Model_TE_aug_decoders, custom_build

In [3]:
n_partitions=40
cvfold=0

D = load_bioarxiv_dataset('../data/proc/')
XT = D['T_dat']
XE = D['E_pcipfx']
XE[np.isnan(XE)] = 0.0

ind_dict = partitions(D['cluster'], n_partitions=40, seed=0)
train_ind,val_ind = ind_dict[cvfold]['train'],ind_dict[cvfold]['val']

T_dim = XT.shape[1]
E_dim = XE.shape[1]

In [4]:
#Load and build model
cplAE = Model_TE_aug_decoders(T_dim,E_dim)
cplAE = custom_build(cplAE,(T_dim,E_dim))

#Load pre-trained network weights
cplAE.load_weights('../data/trained_models/NM_Edat_pcipfx_aT_1-0_aE_1-0_cs_1-0_ad_1_ld_3_bs_200_se_500_ne_1500_cv_0_ri_11_500_ft-weights.h5')

Within modality reconstructions: 
 - $X_{t}\rightarrow z_{t}\rightarrow \widetilde{X}_t$
 - $X_{e}\rightarrow z_{e}\rightarrow \widetilde{X}_e$

Cross-modal reconstructions:
 - $X_{t}\rightarrow z_{t}\rightarrow \widetilde{X}_e$
 - $X_{e}\rightarrow z_{e}\rightarrow \widetilde{X}_t$

In [5]:
#Use only the pre-trained encoders to get representation
zT = cplAE.encoder_T(tf.constant(XT,dtype=tf.float32),training=False)
zE = cplAE.encoder_E(tf.constant(XE,dtype=tf.float32),training=False)

#Use pre-trained decoders to obtain reconstruction of the same modality as the input
XrT = cplAE.decoder_T(zT,training=False)
XrE = cplAE.decoder_E(zE,training=False)

#Use pre-trained decoder of the 'other' modality to get cross modal reconstruction
XrT_cm = cplAE.decoder_T(zE,training=False)
XrE_cm = cplAE.decoder_E(zT,training=False)

In [6]:
#Convert tensors to numpy arrays
zE = zE.numpy()
zT = zT.numpy()

XrT = XrT.numpy()
XrE = XrE.numpy()

XrT_cm = XrT_cm.numpy()
XrE_cm = XrE_cm.numpy()

Within modality reconstructions can be viewed as an upper bound for accuracy of the cross modal reconstructions